<a href="https://colab.research.google.com/github/lacls/Aspect-Based-Sentiment-Analysis/blob/master/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Library installation*

In [ ]:
!pip install underthesea
!pip install pyvi
import numpy as np
from numpy import random
import random as rn
random.seed(12345)
import tensorflow as tf
np.random.seed(12345)
rn.seed(12345)
from underthesea import word_tokenize,  pos_tag
import os, pickle, re, keras, sklearn, string
from pyvi import ViTokenizer, ViPosTagger
from transformers import AutoModel, AutoTokenizer
from keras.preprocessing.sequence import pad_sequences

     |████████████████████████████████| 7.5MB 7.8MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 753.2MB 23kB/s 
     |████████████████████████████████| 1.3MB 38.1MB/s 
     |████████████████████████████████| 245kB 50.1MB/s 
     |████████████████████████████████| 747kB 34.4MB/s 
     |████████████████████████████████| 890kB 36.5MB/s 
     |████████████████████████████████| 2.9MB 48.3MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=6dbe6b0410be0dff867f6ee9f3d855eafc939fc5816b015a24495c584231f34b
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6c783e6ccae978233448f0fe7df79ac71a7fbadbcc31c9eb76d5b95d0f796d43
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123

# Data Configuration

## Read file

In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="str")

# Optional: if you want to clone a github repository
#init_git_cloudflared(githubRepositoryUrl)

## Library Install

## AVIVN

> Notice: positive is 0, negative is 1, according the webpage

In [ ]:
idx_=[]
text=[]
label=[]
import collections
##AVIVN

with open("/content/drive/MyDrive/Thesis/Data/SA_demo/train.txt","r") as f:
    bla=f.readlines()

store_text=""
idx_=[]
text=[]
label=[]
for value in bla:
  value=value.replace("\n","")
  if value:
    if "train" in value:
      if store_text:
        text.append(store_text)
        store_text=""
        if len(text)!=len(label):
          print(value)
          break
      idx_.append(value)
    elif collections.Counter(value)['"']==2:
      text.append(value.replace('"',""))
    elif value in ["0","1","2"]:
      label.append(value)
    else:
      value=value.replace('"',"")
      store_text+=f"{value} "

data_csv=pd.DataFrame(data={'text':text,'label':label})
data_csv.to_csv("/content/drive/MyDrive/Thesis/Data/aivivn_train.csv",index=False)

In [ ]:
import collections
##AVIVN

with open("/content/drive/MyDrive/Thesis/Data/SA_demo/test.txt","r") as f:
    bla=f.readlines()

store_text=""
idx_=[]
text=[]
label=[]
for value in bla:
  value=value.replace("\n","")
  if value:
    if "test_" in value:
      if store_text:
        text.append(store_text)
        store_text=""
        if len(text)!=len(idx_):
          break
      idx_.append(value)
    elif collections.Counter(value)['"']==2:
      text.append(value.replace('"',""))
    # elif value in ["0","1","2"]:
    #     label.append(value)
    else:
      value=value.replace('"',"")
      store_text+=f"{value} "

data_csv=pd.DataFrame(data={'idx_':idx_,'text':text})
data_csv.to_csv("/content/drive/MyDrive/Thesis/Data/aivivn_test.csv",index=False)

## KSE

> Notice: Neutral is labeled 2, Negative is 0 and Positive is 1

In [ ]:
idx_=[]
text=[]
label=[]
import collections
import os
sentiment_Value=0
for value in os.listdir('/content/drive/MyDrive/Thesis/Data/KSE/train/'):
  with open(f"/content/drive/MyDrive/Thesis/Data/KSE/train/{value}","r") as f:
      data=f.readlines()
      if "_0" in value:
        sentiment_Value=0
      elif "_1" in value:
        sentiment_Value=1
      else:
        sentiment_Value=2
      for h in data:
        text.append(h)
        label.append(sentiment_Value)

data_csv=pd.DataFrame(data={'text':text,'label':label})
data_shuffle=data_csv.sample(frac=1).reset_index(drop=True)
data_shuffle.to_csv("/content/drive/MyDrive/Thesis/Data/kse_train.csv",index=False)     
  
idx_=[]
text=[]
label=[]
import collections
import os
sentiment_Value=0
for value in os.listdir('/content/drive/MyDrive/Thesis/Data/KSE/test/'):
  with open(f"/content/drive/MyDrive/Thesis/Data/KSE/test/{value}","r") as f:
      data=f.readlines()
      if "_0" in value:
        sentiment_Value=-1
      elif "_1" in value:
        sentiment_Value=1
      else:
        sentiment_Value=0
      for h in data:
        text.append(h)
        label.append(sentiment_Value)
  
data_csv=pd.DataFrame(data={'text':text,'label':label})
data_shuffle=data_csv.sample(frac=1).reset_index(drop=True)
data_shuffle.to_csv("/content/drive/MyDrive/Thesis/Data/kse_test.csv",index=False)     
    

## UIT-VSFC

In [ ]:
import pandas as pd

idx_=[]
text=[]
label=[]
import collections
import os
with open(f"/content/drive/MyDrive/Thesis/Data/UIT-VSFC/train/sents.txt","r") as f:
    text=f.read().splitlines()

with open(f"/content/drive/MyDrive/Thesis/Data/UIT-VSFC/train/sentiments.txt","r") as f:
    label=f.read().splitlines()

data_csv=pd.DataFrame(data={'text':text,'label':label})
data_csv.to_csv("/content/drive/MyDrive/Thesis/Data/uit_TOPICS_vsfc_train.csv",index=False)


with open(f"/content/drive/MyDrive/Thesis/Data/UIT-VSFC/dev/sents.txt","r") as f:
    text=f.read().splitlines()

with open(f"/content/drive/MyDrive/Thesis/Data/UIT-VSFC/dev/sentiments.txt","r") as f:
    label=f.read().splitlines()

data_csv=pd.DataFrame(data={'text':text,'label':label})
data_csv.to_csv("/content/drive/MyDrive/Thesis/Data/uit_TOPICS_vsfc_dev.csv",index=False)

with open(f"/content/drive/MyDrive/Thesis/Data/UIT-VSFC/test/sents.txt","r") as f:
    text=f.read().splitlines()

with open(f"/content/drive/MyDrive/Thesis/Data/UIT-VSFC/test/sentiments.txt","r") as f:
    label=f.read().splitlines()

data_csv=pd.DataFrame(data={'text':text,'label':label})
data_csv.to_csv("/content/drive/MyDrive/Thesis/Data/uit_TOPICS_vsfc_test.csv",index=False)

In [ ]:
data_csv=pd.read_csv("/content/drive/MyDrive/Thesis/Data/uit_TOPICS_vsfc_dev.csv")
data_csv

,text,label
0,giáo trình chưa cụ thể .,0
1,giảng buồn ngủ .,0
2,"giáo viên vui tính , tận tâm .",2
3,"giảng viên nên giao bài tập nhiều hơn , chia n...",0
4,"giảng viên cần giảng bài chi tiết hơn , đi sâu...",0
...,...,...
1578,hướng dẫn lab mơ hồ .,0
1579,thầy cho chúng em những bài tập mang tính thực...,2
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0


## VLSP

In [ ]:
import pandas as pd
import numpy as np

text=[]
sentiment=[]

with open("/content/drive/MyDrive/Thesis/Data/VLSP 2016/SA2016-training_data/SA-training_positive.txt","r") as f:
    bla=f.read().splitlines() 
    k=0
    for h in bla:
      if h!="":
        text.append(h)
        k+=1
    for _ in range(k):
      sentiment.append(1)
with open("/content/drive/MyDrive/Thesis/Data/VLSP 2016/SA2016-training_data/SA-training_neutral.txt","r") as f:
    bla=f.read().splitlines() 
    k=0
    for h in bla:
      if h!="":
        text.append(h)
        k+=1
    for _ in range(k):
      sentiment.append(-1)
with open("/content/drive/MyDrive/Thesis/Data/VLSP 2016/SA2016-training_data/SA-training_negative.txt","r") as f:
    bla=f.read().splitlines() 
    k=0
    for h in bla:
      if h!="":
        text.append(h)
        k+=1
    for _ in range(k):
      sentiment.append(0)



data_csv=pd.DataFrame(data={'text':text,'sentiment':sentiment})
data_shuffle=data_csv.sample(frac=1).reset_index(drop=True)

data_shuffle.to_csv("/content/drive/MyDrive/Thesis/Data/vlsp_train.csv",index=False)

In [ ]:
text=[]
sentiment=[]
with open("/content/drive/MyDrive/Thesis/Data/VLSP 2016/SA2016-TestData-Ans/test_raw_ANS.txt","r") as f:
    bla=f.read().splitlines() 
    for index, value in enumerate(bla):
      if index%2==0:
        text.append(value)
      else:
        if value=="POS":
          sentiment.append(1)
        elif value=="NEU":
          sentiment.append(0)
        else:
          sentiment.append(-1)

data_csv=pd.DataFrame(data={'text':text,'sentiment':sentiment})
data_shuffle=data_csv.sample(frac=1).reset_index(drop=True)
data_shuffle.to_csv("/content/drive/MyDrive/Thesis/Data/vlsp_test.csv",index=False)

## HOTEL

In [ ]:
import re
import pandas as pd
sentiment=[]
text=[]
datContent = [i for i in open("/content/drive/MyDrive/Thesis/Data/data1Wordalltrain.dat").readlines()]
for h in datContent:
  string=""
  for value in h.split(" "):
    if "NEGA" in value:
      sentiment.append(-1)
    elif "POS" in value:
      sentiment.append(1)
    elif "NEU" in value:
      sentiment.append(0)
    else:
      value_=re.sub("[\/].*$","",value)
      string.join(f'{value_} ')
  text.append(string)
  if len(sentiment)!=len(text):
    break
    print(string)

data_csv=pd.DataFrame(data={'text':text,'sentiment':sentiment})
data_shuffle=data_csv.sample(frac=1).reset_index(drop=True)

data_shuffle.to_csv("/content/drive/MyDrive/Thesis/Data/hotel.csv",index=False)

# Data Augmentation

### **Word_2Vec replacement**
- For each word in a tweet, we query the exter- nal embeddings, and replace them with their knn words to create a new training instance. For ex- ample, consider the tweet “Being late is terrible” with the punctuality label, after searching for knn words for each token, we create a new training in- stance: “Be behind are bad” with the same label

- IMPORTANT, This can lead to out-of-context and unnat- urally complex replacements 

#### *Clean_doc*

In [ ]:
def normalText(sent):
    sent = str(sent).replace('_',' ').replace('/',' trên ')
    sent = re.sub('-{2,}','',sent)
    sent = re.sub('\\s+',' ', sent)
    patPrice = r'([0-9]+k?(\s?-\s?)[0-9]+\s?(k|K))|([0-9]+(.|,)?[0-9]+\s?(triệu|ngàn|trăm|k|K|))|([0-9]+(.[0-9]+)?Ä‘)|([0-9]+k)'
    patHagTag = r'#\s?[aăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ]+'
    patURL = r"(?:http://|www.)[^\"]+"
    sent = re.sub(patURL,'website',sent)
    sent = re.sub(patHagTag,' hagtag ',sent)
    sent = re.sub(patPrice, ' giá_tiền ', sent)
    sent = re.sub('\.+','.',sent)
    sent = re.sub('(hagtag\\s+)+',' hagtag ',sent)
    sent = re.sub('\\s+',' ',sent)
    return sent


def normalize_elonge_word(sent):
    s_new = ''
    for word in sent.split(' '):
        word_new = ''
        for char in word.strip():
            if char != word_new[-1]:
                word_new += char
    s_new += word_new.strip() + ' '
    return s_new

def tokenizer(text):
      token = word_tokenize(text,format="text")
      token = token.replace('giá tiền','giá_tiền').replace('Giá tiền','Giá_tiền')
      return token

def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()

def normalize_elonge_word(sent):
    s_new = ''
    for word in sent.split(' '):
        word_new = ' '
        for char in word.strip():
            if char != word_new[-1]:
                word_new += char
        s_new += word_new.strip() + ' '
    return s_new.strip()  

def clean_doc(doc,set_work=False):
    for punc in string.punctuation:
        doc = doc.replace(punc,' '+ punc + ' ')
    doc = normalText(doc)
    doc = deleteIcon(doc)
    if set_work:
      doc = tokenizer(doc)
    # Lowercase
    doc = doc.lower()
    # Removing multiple whitespaces
    doc = re.sub(r"\?", " \? ", doc)
    # Remove numbers
    doc = re.sub(r"[0-9]+", " num ", doc)
    # Split in tokens
    # Remove punctuation
    for punc in string.punctuation:
      if punc not in "_":
          doc = doc.replace(punc,' ')
    doc = re.sub('\\s+',' ',doc)
    doc = normalize_elonge_word(doc)
    return doc

#### *Baomoi*

In [ ]:
print(ViPosTagger.postagging(clean_doc(text[0],True)))
print(pos_tag(clean_doc(text[0])))
# https://pypi.org/project/pyvi/
#model_bao_moi.similar_by_word("son",10)

(['dung', 'dc', 'sp', 'tot', 'cam', 'on_shop', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn', 'chất_lượng', 'sản_phẩm', 'tuyệt_vời'], ['N', 'V', 'N', 'V', 'N', 'N', 'N', 'N', 'R', 'A', 'C', 'A', 'N', 'N', 'A'])
[('dung', 'N'), ('dc', 'V'), ('sp', 'N'), ('tot', 'V'), ('cam', 'N'), ('on shop', 'N'), ('đóng gói', 'V'), ('sản phẩm', 'N'), ('rất', 'R'), ('đẹp', 'A'), ('và', 'C'), ('chắc chắn', 'A'), ('chất lượng', 'N'), ('sản phẩm', 'N'), ('tuyệt vời', 'A')]


In [ ]:
model_bao_moi.similar_by_word("ảnh",10)

[('hình', 0.7319726943969727),
 ('hình_hài_hước', 0.6114365458488464),
 ('hình_ảnh', 0.5983822345733643),
 ('screenshot', 0.5368694067001343),
 ('Ảnh_cực_Đã', 0.5335326194763184),
 ('bộ_ảnh', 0.5332967638969421),
 ('tiệc_cực_vui', 0.5284527540206909),
 ('loobook', 0.5211629867553711),
 ('choẹt', 0.5155001282691956),
 ('hình_như_thế', 0.5113717317581177)]

In [ ]:
import gensim
model_bao_moi = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP@Research/Aspect/Embedding/baomoi.bin', binary=True)


#### *fastText*

In [ ]:
from gensim.models.wrappers import FastText
model_ftext = FastText.load_fasttext_format('/content/drive/MyDrive/NLP@Research/Aspect/Embedding/cc.vi.300.bin')
model_ftext.similar_by_word("ai_vậy",10)

### **TF-IDF**
- This augmentation method was proposed by Xie et al. in the Unsupervised Data Augmentation paper. The basic idea is that words that have low TF-IDF scores are uninformative and thus can be replaced without affecting the ground-truth labels of the sentence.

### **Bert-base**

- Transformer models such as BERT, ROBERTA, and ALBERT have been trained on a large amount of text using a pretext task called “Masked Language Modeling” where the model has to predict masked words based on the context.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

tokenizer_ = AutoTokenizer.from_pretrained("vinai/phobert-base")
model_ = AutoModelWithLMHead.from_pretrained("vinai/phobert-base")
nlp = pipeline("fill-mask", model=model_,tokenizer=tokenizer_)
nlp('Đồ ăn <mask>, mình thích lắm ')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


[{'score': 0.07473760843276978,
  'sequence': '<s> Đồ ăn Việt_Nam, mình thích lắm </s>',
  'token': 56,
  'token_str': 'Việt_Nam'},
 {'score': 0.06641945987939835,
  'sequence': '<s> Đồ ăn vặt, mình thích lắm </s>',
  'token': 5296,
  'token_str': 'vặt'},
 {'score': 0.05321295186877251,
  'sequence': '<s> Đồ ăn Nhật, mình thích lắm </s>',
  'token': 1219,
  'token_str': 'Nhật'},
 {'score': 0.051812101155519485,
  'sequence': '<s> Đồ ăn Hàn_Quốc, mình thích lắm </s>',
  'token': 536,
  'token_str': 'Hàn_Quốc'},
 {'score': 0.0393323078751564,
  'sequence': '<s> Đồ ăn nhẹ, mình thích lắm </s>',
  'token': 1061,
  'token_str': 'nhẹ'}]

In [ ]:
import torch
sequence = f" Đồ ăn {tokenizer.mask_token} thích lắm."

input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model_(input)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

 Đồ ăn yêu thích lắm.
 Đồ ăn bạn thích lắm.
 Đồ ăn tôi thích lắm.
 Đồ ăn không thích lắm.
 Đồ ăn Ý thích lắm.


In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch



sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."

input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))